In [3]:
# !pip3 install torch
# !pip install scikit-learn

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [5]:
# get data
data = pd.read_csv("../usa/processed-usa-data.csv", delimiter=",", names=["Temperature_Bin", "Wind_Speed_Bin", "Amount_of_Accidents"], skiprows=1)
data = data.dropna()

In [6]:
from sklearn.model_selection import train_test_split

# Preprocess the data
data = data.dropna()

accidents = data["Amount_of_Accidents"].values
accidents = accidents.sum() / len(accidents)

print("average accidents: ", accidents)

# Perform one-hot encoding on the categorical columns
temperature_encoded = pd.get_dummies(data["Temperature_Bin"], prefix="temperature")
wind_speed_encoded = pd.get_dummies(data["Wind_Speed_Bin"], prefix="wind_speed")

# Combine the encoded features with the amount of accidents
processed_data = pd.concat([temperature_encoded, wind_speed_encoded, data["Amount_of_Accidents"]], axis=1)

# Convert the processed data to PyTorch tensors
X = processed_data.iloc[:, :-1].values
y = processed_data.iloc[:, -1].values

X_train, x_val, y_train,  y_val = train_test_split(X, y, test_size=0.2, shuffle=True)
 
X.shape
X_train.shape, x_val.shape

y_train
# Standardize the input features
# X_mean = torch.mean(X, dim=0)
# X_std = torch.std(X, dim=0)
# X_standardized = (X - X_mean) / X_std


# X.size(), y.size()

average accidents:  20240.4


array([     5,    717,     57,     99, 162833,     29,     72,     17,
         1915,    534, 140630,  11877,   2594,  85158,   8282,  62352,
          223,    565,    133,      0,   2987,   2858, 100558,     35,
          310,    146,     74,  15178,  51595, 111974,   1642,    507,
           81,   6050,     28,    565,    182,   1213,  34356,  10728,
         8927, 181655,     19,      0,    173,  50086, 136080,  25962,
        24186,     45,   2018,   3503,  21627,     75, 108061,    163,
         3273,   9394,  10643,  56383,   5175,    122,   9247,  32693,
        27068,    830,   4713,  15802,     52,   7546,  10247,      7,
        46526,    414,   9824,  22582,   3048,    257,  38569,      3])

In [8]:
# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(X_standardized.shape[1], 1)  # Number of input features and 1 output

    def forward(self, x):
        return self.linear(x)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

lr = LinearRegression()
lr.fit(X_train, np.log(y_train + 1e-5))

rf = RandomForestRegressor()
rf.fit(X_train, np.log(y_train + 1e-5))

# save modle
torch.save(rf, "usa-model.pt")


In [ ]:
lr.score(x_val, np.log(y_val + 1e-5)),rf.score(x_val, np.log(y_val + 1e-5))

(0.45312342691723884, 0.9372714312721295)

In [ ]:
# Define the bins for temperature and wind speed
temperature_bins = [-float("inf"), -5, 0, 5, 10, 15, 20, 25, 30, 35, float("inf")]
wind_speed_bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, float("inf")]

# Assign labels to each bin
temperature_labels = [
    "Extremely Cold",
    "Very Cold",
    "Cold",
    "Slightly Cold",
    "Moderate Cold",
    "Moderate Hot",
    "Slightly Hot",
    "Hot",
    "Very Hot",
    "Extremely Hot",
]
wind_speed_labels = [
    "No Wind",
    "Very Low",
    "Low",
    "Slightly Low",
    "Moderate Low",
    "Moderate High",
    "Slightly High",
    "High",
    "Very High",
    "Extremely High",
]

current_temperature = 10
current_wind_speed = 10

# Encode the current temperature and wind speed values into one-hot encoding
temperature_index = next(i for i, bin in enumerate(temperature_bins) if current_temperature <= bin)
wind_speed_index = next(i for i, bin in enumerate(wind_speed_bins) if current_wind_speed <= bin)

encoded_temperature = [0] * len(temperature_labels)
encoded_temperature[temperature_index] = 1

encoded_wind_speed = [0] * len(wind_speed_labels)
encoded_wind_speed[wind_speed_index] = 1

# Combine the encoded values into a single input tensor
input_tensor = torch.tensor([encoded_temperature + encoded_wind_speed], dtype=torch.float32)

# Make the prediction using the trained model
predicted = model(input_tensor)

# Print the predicted amount of accidents
print(f"Predicted amount of accidents: {predicted.item():.2f}")
deviation = (predicted.item() - 20240.4) / 20240.4 * 100
print("percent deviation from average accidents per month: ", str(round(deviation, 3)) + "%")


Predicted amount of accidents: 20240.31
percent deviation from average accidents per month:  -0.0%
